# LSTM-BI-LSTM-GRU-BI-GRU-RNN-BRNN 
# ALL Optimizers
# SpamBase Dataset

In [1]:
!pip3 install tensorflow-addons
!pip3 install keras-adabound
!pip3 install adabelief-tf==0.2.0

     |████████████████████████████████| 1.1 MB 5.4 MB/s 
  Created wheel for keras-adabound: filename=keras_adabound-0.6.0-py3-none-any.whl size=6607 sha256=c1202386efd682f9ddf070430565ec5801d0789563b2fafc162c05533f5962f7
  Stored in directory: /root/.cache/pip/wheels/3e/bf/39/3d95847ef12aa58c15a6cc7a20f4f21ea39fcd52793e1beea0
Successfully built keras-adabound


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Time and counters
from time import perf_counter

# Optimizers
import tensorflow as tf 
import tensorflow_addons as tfa
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad, Nadam, Adadelta, Adamax, Ftrl
from adabelief_tf import AdaBeliefOptimizer
from keras_adabound import AdaBound

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Lambda, Dense, Embedding, Bidirectional,concatenate
from tensorflow.keras.layers import Dropout, Input,InputLayer, ReLU, LSTM
from tensorflow.keras.layers import GRU, SimpleRNN
# from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#metrics
from sklearn.metrics import f1_score , recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_curve, auc


import warnings
warnings.filterwarnings('ignore')

#Mark Down print
from IPython.display import Markdown, display

def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Optimizers and other paths

In [4]:
#paths
SPAMBASE   = "drive/MyDrive/Mahboob/Datasets/SpamBase/PROCESSED.csv" 
heatmaps   = "drive/MyDrive/Mahboob/Heatmaps/SpamBase/"
Visuals    = "drive/MyDrive/Mahboob/Visuals/SPAMBASE/"
Comparison = "drive/MyDrive/Mahboob/Comparison/"


adabelief      = "drive/MyDrive/Mahboob/Models/SPAMBASE/ADABELIEF/"
adabound      = "drive/MyDrive/Mahboob/Models/SPAMBASE/ADABOUND/"
adadelta      = "drive/MyDrive/Mahboob/Models/SPAMBASE/ADADELTA/"
adagrad      = "drive/MyDrive/Mahboob/Models/SPAMBASE/ADAGRAD/"
adam      = "drive/MyDrive/Mahboob/Models/SPAMBASE/ADAM/"
adamax      = "drive/MyDrive/Mahboob/Models/SPAMBASE/ADAMAX/"
adamw      = "drive/MyDrive/Mahboob/Models/SPAMBASE/ADAMW/"
amsbound      = "drive/MyDrive/Mahboob/Models/SPAMBASE/AMSBOUND/"
amsgrad      = "drive/MyDrive/Mahboob/Models/SPAMBASE/AMSGRAD/"
ftrl      = "drive/MyDrive/Mahboob/Models/SPAMBASE/FTRL/"
nadam      = "drive/MyDrive/Mahboob/Models/SPAMBASE/NADAM/"
radam      = "drive/MyDrive/Mahboob/Models/SPAMBASE/RADAM/"
rmsprop      = "drive/MyDrive/Mahboob/Models/SPAMBASE/RMSPROP/"
sgd      = "drive/MyDrive/Mahboob/Models/SPAMBASE/SGD/"
sgd_nesterov      = "drive/MyDrive/Mahboob/Models/SPAMBASE/SGD-NESTEROV/"
sgd_momentum      = "drive/MyDrive/Mahboob/Models/SPAMBASE/SGD-MOMENTUM/"

In [5]:
# 1. ALL Optimizer List
# Adam weight decay

AdamW = tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.001)
AMSGrad = Adam(amsgrad=True)
Rectified_Adam = tfa.optimizers.RectifiedAdam(learning_rate=0.001)

SGD_momentum = SGD(momentum=0.9)
Nesterov_SGD_momentum = SGD(momentum=0.9, nesterov=True)
#Adam #Nadam #SGD #RMSprop #Adadelta #Adagrad #Adamax #Ftrl
AdaBelief = AdaBeliefOptimizer(learning_rate=1e-3, epsilon=1e-14, rectify=False)
Adabound = AdaBound(lr=1e-3, final_lr=0.1)
AMSbound = AdaBound(lr=1e-3, final_lr=0.1, amsgrad=True)

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.0)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



In [6]:
# drive.mount("/content/drive", force_remount=True)
import sys
sys.path.append("drive/MyDrive/Mahboob/")
!ls
from utils import *

drive  sample_data
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.0)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



# Read Data

In [7]:
data = pd.read_csv(SPAMBASE)
Label = data.label.copy()
data = data.drop(columns="label")
# data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
# data.to_csv(ENRON, index=False)

In [8]:
data.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,word_freq_receive,word_freq_will,word_freq_people,word_freq_report,word_freq_addresses,word_freq_free,word_freq_business,word_freq_email,word_freq_you,word_freq_credit,word_freq_your,word_freq_font,word_freq_000,word_freq_money,word_freq_hp,word_freq_hpl,word_freq_george,word_freq_650,word_freq_lab,word_freq_labs,word_freq_telnet,word_freq_857,word_freq_data,word_freq_415,word_freq_85,word_freq_technology,word_freq_1999,word_freq_parts,word_freq_pm,word_freq_direct,word_freq_cs,word_freq_meeting,word_freq_original,word_freq_project,word_freq_re,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
0,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,0.21,0.79,0.65,0.21,0.14,0.14,0.07,0.28,3.47,0.00,1.59,0.0,0.43,0.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
1,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,0.38,0.45,0.12,0.00,1.75,0.06,0.06,1.03,1.36,0.32,0.51,0.0,1.16,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.06,0.0,0.0,0.12,0.0,0.06,0.06,0.0,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259
2,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0.00,0.00,0.31,0.00,0.00,3.18,0.00,0.31,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191
4,0.00,0.00,0.00,0.0,1.85,0.00,0.00,1.85,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.223,0.0,0.000,0.000,0.000,3.000,15,54


In [9]:
data.describe()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,word_freq_receive,word_freq_will,word_freq_people,word_freq_report,word_freq_addresses,word_freq_free,word_freq_business,word_freq_email,word_freq_you,word_freq_credit,word_freq_your,word_freq_font,word_freq_000,word_freq_money,word_freq_hp,word_freq_hpl,word_freq_george,word_freq_650,word_freq_lab,word_freq_labs,word_freq_telnet,word_freq_857,word_freq_data,word_freq_415,word_freq_85,word_freq_technology,word_freq_1999,word_freq_parts,word_freq_pm,word_freq_direct,word_freq_cs,word_freq_meeting,word_freq_original,word_freq_project,word_freq_re,word_freq_edu,word_freq_table,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
count,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000,4600.000000
mean,0.104576,0.212922,0.280578,0.065439,0.312222,0.095922,0.114233,0.105317,0.090087,0.239465,0.059837,0.541680,0.093950,0.058639,0.049215,0.248833,0.142617,0.184504,1.662041,0.085596,0.809728,0.121228,0.101667,0.094289,0.549624,0.265441,0.767472,0.124872,0.098937,0.102874,0.064767,0.047059,0.097250,0.047846,0.105435,0.097498,0.136983,0.013204,0.078646,0.064848,0.043676,0.132367,0.046109,0.079213,0.301289,0.179863,0.005446,0.031876,0.038583,0.139061,0.016980,0.268960,0.075827,0.044248,5.191827,52.170870,283.290435
std,0.305387,1.290700,0.504170,1.395303,0.672586,0.273850,0.391480,0.401112,0.278643,0.644816,0.201565,0.861791,0.301065,0.335219,0.258871,0.825881,0.444099,0.530930,1.775669,0.509821,1.200938,1.025866,0.350321,0.442681,1.671511,0.887043,3.367639,0.538631,0.593389,0.456729,0.403435,0.328594,0.555966,0.329480,0.532315,0.402664,0.423493,0.220675,0.434718,0.349953,0.361243,0.766900,0.223835,0.622042,1.011787,0.911214,0.076283,0.285765,0.243497,0.270377,0.109406,0.815726,0.245906,0.429388,31.732891,194.912453,606.413764
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.310000,0.000000,0.220000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [10]:
len(data.columns)

57

# Data Splitting

In [11]:
#Splitting the data - 80:20 ratio
X_train, X_test, y_train, y_test = train_test_split(data , Label, test_size = 0.2, random_state = 99)
print(f"Training split input: {X_train.shape}")
print(f"Testing split input : {X_test.shape}")
print(f"Training split class: {y_train.shape}")
print(f"Testing split class : {y_test.shape}")


Training split input: (3680, 57)
Testing split input : (920, 57)
Training split class: (3680,)
Testing split class : (920,)


In [38]:
X_train.shape

(3680, 57)

In [39]:
X_train_reshaped = X_train.to_numpy().reshape((1,X_train.shape[0], X_train.shape[1]))
# X_train_reshaped = np.expand_dims(X_train, -1)
X_train_reshaped.shape

(1, 3680, 57)

In [40]:
X_test.shape

(920, 57)

In [41]:
# X_test_reshaped = np.expand_dims(X_test, -1)
X_test_reshaped = X_test.to_numpy().reshape((1,X_test.shape[0], X_test.shape[1]))
X_test_reshaped.shape

(1, 920, 57)

In [42]:
model = Sequential()
model.add(Bidirectional(LSTM(100, input_shape=(3680,57))))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',])

In [43]:
model.summary()

ValueError: ignored

In [45]:
history = model.fit(X_train_reshaped, y_train, epochs=50)

ValueError: ignored

# Model Training

In [19]:
def Model_Train(arc_func, optim_func):
  model = arc_func(3680, numFeatures=57)
  model.compile(optimizer=optim_func, loss='binary_crossentropy', metrics=['accuracy',])
  start = perf_counter()
  history = model.fit(X_train_reshaped, y_train, batch_size=512, epochs=10, validation_split=0.2)
  duration = perf_counter() - start
  duration = round(duration,2)

  # Predicting the Test set results
  y_pred = model.predict(X_test_reshaped)
  y_pred = (y_pred > 0.5)
  y_pred = np.array(y_pred)

  test_loss, test_acc = model.evaluate(X_test_reshaped, y_test)
  test_err = 100 - test_acc*100

  print(f"Test Loss:     {test_loss*100} %")
  print(f"Test Accuracy: {test_acc*100}  %")
  print(f"Test error: {test_err}  %")

  return model, history, duration, y_pred, test_loss

In [20]:
Optimizer_list = ["Adam", "Nadam", "SGD", "RMSProp", "Adagrad", "Adadelta", "Ftrl", "SGD-Momentum", "SGD-Nesterov-Momentum", "Adam-Weight-Decay"] 

Optimizer_function_list = [Adam(), Nadam(), SGD(), RMSprop(), Adagrad(), Adadelta(), Ftrl(), SGD(momentum=0.9), SGD(momentum=0.9, nesterov=True), tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.001)]


Model_Architectures = {
    """
    Change model Architectures to 1D models like CNN1D, Dense models (ANN), ML-Classifiers etc
    """ 
    "BRNN" : BI_RNN_ARCHITECTURE_SPAMBASE,
    "Bi-LSTM" : BI_LSTM_ARCHITECTURE_SPAMBASE,
    "Bi-GRU" : BI_GRU_ARCHITECTURE_SPAMBASE
}

Deliverables = {
    "Optimizer_history" : {
          "BRNN" : [],
          "Bi-LSTM" : [],
          "Bi-GRU" : []
    },
    "Models" : {
          "BRNN" : [],
          "Bi-LSTM" : [],
          "Bi-GRU" : []        
    },
    "Duration" : {
          "BRNN" : [],
          "Bi-LSTM" : [],
          "Bi-GRU" : []        
    },
    "Y_pred" : {
          "BRNN" : [],
          "Bi-LSTM" : [],
          "Bi-GRU" : []        
    },
    "Test_Loss" : {
          "BRNN" : [],
          "Bi-LSTM" : [],
          "Bi-GRU" : []        
    },
}



# BRNN - Bi-LSTM - Bi-GRU
## Adam, Nadam, SGD, RMSprop, Adagrad, Adadelta, Ftrl, SGD momentum, SGD Nesterov Momentum, Adamw 

In [21]:
for k,v in enumerate(Model_Architectures.items()):
  for j in Optimizer_function_list:
    print(f"{k} : {v} , optimizer : {j}")
    model, history, duration, y_pred, test_loss = Model_Train(v[1],j)
    Deliverables["Optimizer_history"][v[0]].append(history)
    Deliverables["Models"][v[0]].append(model)
    Deliverables["Duration"][v[0]].append(duration)
    Deliverables["Y_pred"][v[0]].append(y_pred)
    Deliverables["Test_Loss"][v[0]].append(test_loss)
    print("done training")

0 : ('BRNN', <function BI_RNN_ARCHITECTURE_SPAMBASE at 0x7f26733088c0>) , optimizer : <keras.optimizer_v2.adam.Adam object at 0x7f26731101d0>


ValueError: ignored

# Model comparison table with metrics : 
1. Accuracy
  - Train Accuracy from history object
  - Test Accuracy from get_Metrics Function in utils
2. Loss
  - Train loss curve From history object
  - Test loss From Model Evaluation  
3. Precision, Recall, F1 score
4. ROC AUC

In [ ]:
"""
Model comparison table with metrics : 
1. Accuracy
2. Loss
3. Error
4. Precision, Recall, F1 score
5. ROC AUC
"""
Model_comparison = dict()

for k,v in enumerate(Model_Architectures.items()):
  for index_of_optim, i in enumerate(Optimizer_list):
    # print(v[0])
    history_curr = Deliverables["Optimizer_history"][v[0]][index_of_optim]
    acc_train = history_curr.history["accuracy"][len(history_curr.history["accuracy"])-1]
    
    y_pred = Deliverables["Y_pred"][v[0]][index_of_optim]
    
    loss_test = Deliverables["Test_Loss"][v[0]][index_of_optim]
    
    duration = Deliverables["Duration"][v[0]][index_of_optim]
    
    precision, recall, f1_score_, acc_test, auc_ = get_Metrics(Y_test, y_pred)
    
    Model_comparison[f"{v[0]} {i}"] = { 
        "Train_acc":acc_train, 
        "Test_acc":acc_test, 
        "Test_loss" : loss_test, 
        "auroc":auc_, 
        "f1_score":f1_score_, 
        "precision":precision, 
        "recall":recall,  
        "duration": duration 
        }
    print(f"{v[0]} {i} done")    

In [ ]:
import re
models_metrics = []
BRNN_metrics = []
BILSTM_metrics = []
BIGRU_metrics = []
ALL = [models_metrics, BRNN_metrics, BILSTM_metrics, BIGRU_metrics] 
ALL_names = ["SpamBase_comparison", "SpamBase_BRNN_comparison", "SpamBase_Bi-LSTM_comparison", "SpamBase_Bi-GRU_comparison"]

for name, model in Model_comparison.items():
  precision, recall, f1_score_,  = model["precision"], model["recall"], model["f1_score"]
  Test_acc, auc_, Test_loss = model["Test_acc"], model["auroc"], model["Test_loss"],
  Train_acc, duration = model["Train_acc"], model["duration"]
  models_metrics.append([name, precision, recall, f1_score_, Train_acc, Test_acc, Test_loss, auc_, duration])
  if re.match(r"^BRNN*", name):
    BRNN_metrics.append([name, precision, recall, f1_score_, Train_acc, Test_acc, Test_loss, auc_, duration])
  if re.match(r"^Bi-LSTM*", name):
    BILSTM_metrics.append([name, precision, recall, f1_score_, Train_acc, Test_acc, Test_loss, auc_, duration])
  if re.match(r"^Bi-GRU*", name):
    BIGRU_metrics.append([name, precision, recall, f1_score_, Train_acc, Test_acc, Test_loss, auc_, duration])

for i,j in zip(ALL, ALL_names):
  df_metrics = pd.DataFrame(i)
  df_metrics.columns = ['Model', 'Precision', 'Recall', 'f1 score', "Train Accuracy", 'Test Accuracy', 'Loss', 'ROC-AUC', 'Train Time (s)']
  df_metrics.sort_values(by = 'Test Accuracy', ascending = False, inplace=True)
  df_metrics.reset_index(drop = True, inplace=True)
  df_metrics.to_csv(f"{Comparison}{j}.csv")

In [ ]:
import pprint
pprint.pprint(Model_comparison)

In [ ]:
df_metrics

# Save the model objects for graph

In [ ]:
import pickle
with open(f"{Visuals}SpamBase_history.pkl", "wb") as open_file:
    pickle.dump(Deliverables["Optimizer_history"], open_file)
with open(f"{Visuals}SpamBase_y_pred.pkl", "wb") as open_file:
    pickle.dump(Deliverables["Y_pred"], open_file)
with open(f"{Visuals}SpamBase.pkl", "wb") as open_file:
    pickle.dump(Deliverables["Y_pred"], open_file)

In [ ]:
Histories = Deliverables["Optimizer_history"].copy()

col = ['#feb24c','#fd8d3c','#fc4e2a','#e31a1c','#bd0026','#dadaeb','#bcbddc','#9e9ac8','#807dba','#6a51a3']
col_2 = ["#238b45", "#88419d", "#2b8cbe", "#ef6548", "#3690c0", "#df65b0", "#41b6c4", "#6a51a3", "#c7e9b4", "#74a9cf"]
col_3 = ["#003f5c","#2f4b7c","#665191","#a05195","#d45087","#f95d6a","#ff7c43","#ffa600","#488f31","#bad0af"]

from cycler import cycler
import matplotlib.pyplot as plt
custom_cycler = (cycler(color=col_3) +
                 cycler(marker=['o', '8', 's', 'p', '*', 'h', 'H', '+', 'x', 'D']))

# Global plot parameters
plt.rc('lines', linewidth=3, markerfacecolor='white', markeredgewidth=2, markersize=8, linestyle='--')
plt.rc('axes', prop_cycle=custom_cycler)
font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }

def plot_metric(metric, history, y_label):

  for k, v in enumerate(history.items()):
    print(v[0],v[1])
    fig, ax = plt.subplots(figsize=(7,7))  # Create a figure and an axes.
    #plot legend handles
    plt_handles = []
    for index_of_optim, i in enumerate(Optimizer_list):
      if index_of_optim % 2 == 0:
        plot_ = ax.plot(history[v[0]][index_of_optim].history[metric], linestyle='--', label=i)
        plt_handles.append(plot_[0])
      else: 
        plot_ = ax.plot(history[v[0]][index_of_optim].history[metric], linestyle='-.', label=i)
        plt_handles.append(plot_[0])
    ax.set_xlabel('Epochs', fontdict=font)  
    ax.set_ylabel(y_label, fontdict=font) 
    ax.set_title(f"Spam Base, Model : {v[0]}, {y_label} curve", fontdict=font) 
    plt.savefig(f"{Visuals}Accuracy_Loss/SpamBase_{metric}_{v[0]}.jpeg")
    if k == 0:
      fig_legend = plt.figure()
      plt.legend(handles = plt_handles, loc = 'center', prop={'family': 'serif', 'size': 16}, frameon=False)
      plt.axis("off")
      plt.savefig(f"{Visuals}Accuracy_Loss/SpamBase_{metric}_{v[0]}_legend.jpeg", bbox_inches='tight')


plot_metric("accuracy", Histories, "Train Accuracy")

In [ ]:
plot_metric("val_accuracy", Histories, "Validation Accuracy")

In [ ]:
plot_metric("loss", Histories, "Train Loss")

In [ ]:
plot_metric("val_loss", Histories, "Validation Loss")

In [ ]:
# with open(f"comparison/classifiers/train_test_tf_idf.pkl", "rb") as open_file:
#     dataset_processed = pickle.load(open_file)